In [1]:
import os
import time
import requests
from bs4 import BeautifulSoup

from dotenv import load_dotenv

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

load_dotenv()

True

In [9]:
url = "https://trouverunlogement.lescrous.fr/tools/36/search"

response = requests.get(url)
response

<Response [200]>

In [12]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    search_result = soup.find("h2", class_="SearchResults-desktop")

    if search_result:
        message = search_result.get_text(strip=True)
        if "Aucun logement" not in message:
            aprtment_cards = soup.find_all(
                "li", class_="fr-col-12 fr-col-sm-6 fr-col-md-4 svelte-11sc5my fr-col-lg-4")

            for card in aprtment_cards:
                card_detail = card.find(
                    "p", class_="fr-card__detail fr-icon-group-fill")
                card_colocation_or_individual_text = card_detail.get_text(
                    strip=True)

                if card_colocation_or_individual_text == "Colocation":
                    continue
                elif card_colocation_or_individual_text == "Individuel":
                    apartment_title = card.find(
                        "h3", class_="fr-card__title").get_text(strip=True)
                    print(f"Found an individual appartment, {apartment_title}")

Found an individual appartment, Residence Pascal MARTIN
Found an individual appartment, ROLAND BARTHES
Found an individual appartment, Residence Pascal Paoli 2
Found an individual appartment, Residence Antoine CASANOVA
Found an individual appartment, Residence J.P. Sartre
Found an individual appartment, TANNERIES
Found an individual appartment, Residence Pierrette Grimaldi
Found an individual appartment, RÃ©sidence de Felletin
Found an individual appartment, Residence Farrandu ETTORI
Found an individual appartment, BIDART
Found an individual appartment, Residence Christophe GARELLI
Found an individual appartment, Residence Pierrette Grimaldi
Found an individual appartment, ERLEAK
Found an individual appartment, ANDRE MALRAUX
Found an individual appartment, ARANCETTE
Found an individual appartment, RESIDENCE ROANNE
Found an individual appartment, RESIDENCE CLERISSEAU
Found an individual appartment, MAISON DES ETUDIANTS
Found an individual appartment, JB CLEMENT
Found an individual appar

In [14]:
import traceback


def check_for_new_offers(location, url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        search_result = soup.find("h2", class_="SearchResults-desktop")

        if search_result:
            message = search_result.get_text(strip=True)

            if "Aucun logement" not in message:
                log_content(location, message)
                # send_email(location, message, url, os.getenv("RECEIVER_EMAIL"))
                send_email(location, message, url, "simad3647@gmail.com")


def log_content(location, message):
    log_entry = f"{location.ljust(60)}: {message}\n"
    logger.info(log_entry)


def send_email(location, message, url, receiver_email):
    sender_email = os.getenv("SENDER_EMAIL")
    sender_password = os.getenv("SENDER_PASSWORD")

    subject = "New Apartment Offers"
    body = f"There are new apartment offers in \
        {location}: {message}\nurl : {url}"

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = subject

    msg.attach(MIMEText(body, "plain"))

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            text = msg.as_string()
            server.sendmail(sender_email, receiver_email, text)
            print(f"Email notification sent for {location}")
    except Exception as e:
        print(f"Error sending email: {e}")
        traceback.print_exc()